# Example Notebook 2: A Simple Quench Series
Here we provie a minimal working example of running a series of quenches to different isotropies using parralelized computation. Running this entire notebook on my machine takes about 2 minutes of computation time.

In [2]:
import numpy as np
from simulation_module import SimulationConfig, SimulationState, SimulationRunner, SimulationIO, SimulationVisualizer, AnimationMaker
from laser import Laser
import parralel
from time import time
import os

In [3]:
# Initialize a 650 nm laser object
laser650 = laser650 = Laser(650, [1,0], saturation = 10, detuning = -2 * np.pi * 20 / 2, Gamma = 2 * np.pi * 20)

In [4]:
"""
First we must initialize a simulation configuration. We're also going to check to see if we already have an initial
positions file associated with the desired value of N and gamma.
"""

N = 6 # The number of particles
initial_gamma = 1 # The initial isotropy of interest

# Check that we already have an initial positions file associated with that value of N and gamma
# If not, you need to generate a stable configuration by using something like SimulationState().minimize_energy() and SimulationState().minimize_forces()
filename = f"{N}_{initial_gamma}_positions_1.json"
assert os.path.exists(filename), f"File '{filename}' not found. You need to generate a starting configuration for that value of N, it seems like you don't have one. Try N = 6, gamma = 1 for example."

# Configure our simulation object with 6 ions, and gamma = 1.
config = SimulationConfig(
    N = N,                        # number of ions
    w = 1.0,                      # frequency in the trap harmonic potential
    g = initial_gamma,            # isotropy factor
    m = 137.327,                  # mass in atomic units
    T_mK = 0,                     # temperature in mK
    dt = 10**-3,                  # timestep
    num_steps = int(10**5),     # number of simulation steps
    damping = False,              # damping is turned off for now
    damping_parameter = 1.0,      # since damping is turned off, the strength of the damping parameter is not relevant
    langevin_temperature = False, # langevin_temperature is turned off for now (no random motion)
    lasers=[laser650]             # we only have one laser in the simulation, that is the 650 nm laser
)

In [5]:
# We can save a short textfile that includes the simulation parameters 
config.save_shortform() # By default the filename will be "simulation_details.txt"

In [6]:
"""
Now we'll make a bunch of folders to store the outputs of our quench simulations
"""

quench_trajectory_folder = "quench_folder"
animation_folder = "animation_folder"
animation_output = "animation_output"
temperature_folder = "temperature_folder"
rolling_average_output = "rolling_average_output"

# Make sure folders exist
os.makedirs(quench_trajectory_folder, exist_ok = True)
os.makedirs(animation_folder, exist_ok = True)
os.makedirs(animation_output, exist_ok = True)
os.makedirs(temperature_folder, exist_ok = True)
os.makedirs(rolling_average_output, exist_ok = True)

## Running a series of quenches
Now we're all set to run a series of quenches. Each quench will have a different value of gamma, we will change the simulation value of gamma, then run the simulation, and then save the trajectory to file. Later on, we can load these trajectories back into the system to produce density plots and animations and other things, but first we must run all the simulations.

NOTE: you should take a look at how many cores your computer has. Mine has 8, and so I can safely run num_workers = 7 in the cell below, if however you have many fewer cores, this may really stall your computer

In [7]:
num_quenches = 10 # Number of quenches to be run
num_workers = 7 # This is the number of separate cores on your computer you wish to use in parralel for the computation, for me, 7 is about right, but your machine may have fewer cores
g_start = 0.9 # First value of gamma we will quench to
g_end = 0.75 # Last value of gamma we will quench to

# Now we can run the quenches!
print("This may take a while! (About 45 seconds) on my machine")
start = time()
parralel.run_quench_series(
    config = config,
    loadfile = f"{N}_{config.g}_positions_1.json",
    output_dir = quench_trajectory_folder,
    g_start = g_start,
    g_end = g_end,
    g_step = num_quenches,
    num_workers = num_workers
)    
print(f"Time taken to quench = {time()-start:.3f} s")

This may take a while! (About 45 seconds) on my machine
Beginning to parralel quench
Running gamma = 0.88333333
Running with isotropy (γ) = 0.8833 for 100.0 μs
Running gamma = 0.83333333
Running with isotropy (γ) = 0.8333 for 100.0 μs
Running gamma = 0.80000000
Running with isotropy (γ) = 0.8000 for 100.0 μs
Running gamma = 0.86666667
Running with isotropy (γ) = 0.8667 for 100.0 μs
Running gamma = 0.78333333
Running with isotropy (γ) = 0.7833 for 100.0 μs
Running gamma = 0.90000000
Running with isotropy (γ) = 0.9000 for 100.0 μs
Running gamma = 0.75000000
Running with isotropy (γ) = 0.7500 for 100.0 μs
Running gamma = 0.85000000
Running with isotropy (γ) = 0.8500 for 100.0 μs
Running gamma = 0.76666667
Running with isotropy (γ) = 0.7667 for 100.0 μs
Running gamma = 0.81666667
Running with isotropy (γ) = 0.8167 for 100.0 μs
All quench simulations completed.
Time taken to quench = 37.002 s


## Generating ion density maps from presaved trajectories
Now we can load the trajectory files back in, and generate ion density maps for them

In [8]:
print("This may take a while! (About 16 seconds) on my machine")
start = time()
parralel.generate_density_map_images_from_quench_folder(quench_trajectory_folder, 
                                                        animation_folder, 
                                                        config, 
                                                        num_workers = num_workers,
                                                       square = False,
                                                       extension = "png")
print(f"Time taken to process images = {time()-start:.3f} s")

This may take a while! (About 16 seconds) on my machine
Beginning to parralel process trajectory files
Processing quench_folder/6_0.81666666666667_traj_100000_steps.json
Processing quench_folder/6_0.90000000000000_traj_100000_steps.json
Processing quench_folder/6_0.88333333333333_traj_100000_steps.json
Processing quench_folder/6_0.80000000000000_traj_100000_steps.json
Processing quench_folder/6_0.83333333333333_traj_100000_steps.json
Processing quench_folder/6_0.76666666666667_traj_100000_steps.json
Processing quench_folder/6_0.86666666666667_traj_100000_steps.json
Processing quench_folder/6_0.78333333333333_traj_100000_steps.json
Processing quench_folder/6_0.85000000000000_traj_100000_steps.json
Processing quench_folder/6_0.75000000000000_traj_100000_steps.json
Done!
Time taken to process images = 11.184 s


## Generating animations from presaved ion density images
With the ion density images created, we can generate an animation of them

In [9]:
print("This may take a while! (About 2 seconds) on my machine")
start = time()
AnimationMaker.make_gif_or_mp4_from_images(animation_folder, f"{animation_output}/ion_density_animation_{int(config.num_steps * config.dt)}us.mp4", fps=6, reverse = False)
print(f"Time taken to make animation = {time()-start:.3f} s")

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 600) to (608, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


This may take a while! (About 2 seconds) on my machine
MP4 saved via imageio: animation_output/ion_density_animation_100us.mp4
Time taken to make animation = 0.329 s


## Generating temperature plots
It is often useful to look at the temperature of each simulation over time. Similar to with ion density maps, we can load all of the trajectory files back into the system, compute the average velocity over time (as a proxy for temperature) and plot this over time. Then after creating many such images, we can animate the result

In [10]:
# Generate temperature plots

print("This may take a while! (About 16 seconds) on my machine")
start = time()
parralel.generate_temperature_plots_from_quench_folder(quench_trajectory_folder,
                                                       temperature_folder,
                                                       config, 
                                                       grainyness=100,
                                                       num_workers=num_workers,
                                                       extension = "png"
                                                       )
print(f"Time taken to make temperature plots {time()-start:.5f} s")

This may take a while! (About 16 seconds) on my machine
🔁 Beginning parallel temperature plot generation
Generating temperature plot for quench_folder/6_0.88333333333333_traj_100000_steps.json
Generating temperature plot for quench_folder/6_0.86666666666667_traj_100000_steps.json
Generating temperature plot for quench_folder/6_0.81666666666667_traj_100000_steps.json
Generating temperature plot for quench_folder/6_0.80000000000000_traj_100000_steps.json
Generating temperature plot for quench_folder/6_0.90000000000000_traj_100000_steps.json
Generating temperature plot for quench_folder/6_0.78333333333333_traj_100000_steps.json
Generating temperature plot for quench_folder/6_0.83333333333333_traj_100000_steps.json
Generating temperature plot for quench_folder/6_0.76666666666667_traj_100000_steps.json
Generating temperature plot for quench_folder/6_0.85000000000000_traj_100000_steps.json
Generating temperature plot for quench_folder/6_0.75000000000000_traj_100000_steps.json
✅ All temperatu

In [11]:
# Animate temperature plots
print("This may take a while! (About 2 seconds) on my machine")
start = time()
AnimationMaker.make_gif_or_mp4_from_images(temperature_folder, f"{animation_output}/temperature_animation_{int(config.num_steps * config.dt)}us.mp4", fps=6, reverse = True)
print(f"Time taken to make temperature animation = {time()-start:.3f} s")

This may take a while! (About 2 seconds) on my machine
MP4 saved via imageio: animation_output/temperature_animation_100us.mp4
Time taken to make temperature animation = 0.345 s


## Generating rolling average density maps 
For the length of simulation we have run above, this is a little contrived, but it may be useful to generate rolling average density plots instead of simply one large density plots, here we do that

In [12]:
print("This may take a while! (About 16 seconds) on my machine")

start = time()
parralel.generate_rolling_average_density_map_images_from_quench_folder(
    traj_folder = quench_trajectory_folder,
    output_dir = rolling_average_output,
    base_config = config,
    time_per_image = 33, # Number of microseconds of simulation per image generated
    overlapping_time = 0, # Number of microseconds of overlap of images
    full_histogram = True, # This controls whether or not it also generates a full histogram of the trajectory
    animate = False, # by turning this on you can generate an animation
    num_workers = num_workers,
    extension = "png"
    )
print(f"Time taken to generate rolling images = {time()-start:.3f} s")

This may take a while! (About 16 seconds) on my machine
🔁 Beginning parallel rolling-average density map generation
Saved rolling_average_output/6_0.86666666666667_traj_100000_steps/rolling_frame_002_window_000_gamma_0.866667.png
Saved rolling_average_output/6_0.86666666666667_traj_100000_steps/rolling_frame_002_window_001_gamma_0.866667.png
Saved rolling_average_output/6_0.86666666666667_traj_100000_steps/rolling_frame_002_window_002_gamma_0.866667.png
Saved full density map: rolling_average_output/6_0.86666666666667_traj_100000_steps/rolling_frame_002_FULL_gamma_0.866667.png
Saved rolling_average_output/6_0.88333333333333_traj_100000_steps/rolling_frame_001_window_000_gamma_0.883333.png
Saved rolling_average_output/6_0.88333333333333_traj_100000_steps/rolling_frame_001_window_001_gamma_0.883333.png
Saved rolling_average_output/6_0.88333333333333_traj_100000_steps/rolling_frame_001_window_002_gamma_0.883333.png
Saved full density map: rolling_average_output/6_0.88333333333333_traj_100

## Deleting output folders to restart
You may find it useful to uncomment and use the below script to quickly delete the output files so that you can run the simulation again with new parameters

In [13]:
# import shutil

# # List of folder names to remove
# folders_to_delete = [quench_trajectory_folder, animation_folder, animation_output, temperature_folder, rolling_average_output]  

# # Delete the listed folders
# for folder in folders_to_delete:
#     if os.path.exists(folder) and os.path.isdir(folder):
#         shutil.rmtree(folder)
#         print(f"Deleted folder: {folder}")
#     else:
#         print(f"Folder not found or not a directory: {folder}")

Deleted folder: quench_folder
Deleted folder: animation_folder
Deleted folder: animation_output
Deleted folder: temperature_folder
Deleted folder: rolling_average_output
